<a href="https://colab.research.google.com/github/FaldoFajri/tubes-mesinbelajar/blob/main/Final_Code_Model_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/FaldoFajri/tubes-mesinbelajar.git

Cloning into 'tubes-mesinbelajar'...
remote: Enumerating objects: 2534, done.
remote: Counting objects: 100% (2534/2534), done.
remote: Compressing objects: 100% (2492/2492), done.
remote: Total 2534 (delta 25), reused 2534 (delta 25), pack-reused 0
Receiving objects: 100% (2534/2534), 1.49 GiB | 13.92 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Checking out files: 100% (2603/2603), done.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

%matplotlib inline

In [ ]:
test_dir = "/content/tubes-mesinbelajar/dataset/test"
train_dir = "/content/tubes-mesinbelajar/dataset/train"
val_dir = "/content/tubes-mesinbelajar/dataset/val"

In [ ]:
#DISINI SAYA MENGGUNAKAN MODEL VGG16

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(100,100,3))
base_model.trainable = False

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [ ]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(64, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(2, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    prediction_layer
])

In [ ]:
image_shape = (100,100,3)
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.10,
                               height_shift_range=0.10,
                               rescale=1/255,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_dir,
                                               target_size=(100,100),
                                               batch_size=32,
                                               class_mode='categorical')

Found 1511 images belonging to 2 classes.


In [ ]:
val_image_gen = image_gen.flow_from_directory(val_dir,
                                               target_size=(100,100),
                                               batch_size=32,
                                               class_mode='categorical',shuffle=False)

Found 215 images belonging to 2 classes.


In [ ]:
test_image_gen = image_gen.flow_from_directory(test_dir,
                                               target_size=(100,100),
                                               batch_size=32,
                                               class_mode='categorical',shuffle=False)

Found 432 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
# Compile model
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.000024),
              metrics=['acc'])

#es = EarlyStopping(monitor='val_acc', mode='max', patience=5, restore_best_weights=True)

In [ ]:
results = model.fit(train_image_gen,epochs=100, validation_data=val_image_gen)

Epoch 1/100
48/48 [==============================] - 206s 3s/step - loss: 0.5378 - acc: 0.7340 - val_loss: 0.4804 - val_acc: 0.7349
Epoch 2/100
48/48 [==============================] - 118s 2s/step - loss: 0.4311 - acc: 0.7704 - val_loss: 0.3800 - val_acc: 0.8279
Epoch 3/100
48/48 [==============================] - 135s 3s/step - loss: 0.3504 - acc: 0.8478 - val_loss: 0.3182 - val_acc: 0.9023
Epoch 4/100
48/48 [==============================] - 144s 3s/step - loss: 0.2823 - acc: 0.9272 - val_loss: 0.2632 - val_acc: 0.9209
Epoch 5/100
48/48 [==============================] - 106s 2s/step - loss: 0.2350 - acc: 0.9457 - val_loss: 0.2283 - val_acc: 0.9488
Epoch 6/100
26/48 [===============>..............] - ETA: 43s - loss: 0.2011 - acc: 0.9641

In [ ]:
loss_df = pd.DataFrame(model.history.history)
loss_df

In [ ]:
loss_df[['acc', 'val_acc']].plot()

In [ ]:
loss_df[['loss', 'val_loss']].plot()

In [ ]:
predictions = np.argmax(model.predict(test_image_gen), axis = -1)
predictions
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
print(confusion_matrix(test_image_gen.classes,predictions))
sns.heatmap(confusion_matrix(test_image_gen.classes,predictions), annot = True)